
# **ATENÇÃO**
A lista deverá ser entregue tanto no formato .pdf e em .py ou .ipynb


# **Rotação de uma Imagem**
Uma opção seria utilizar a função de rotação do OpenCV. A ideia do exercício é não utilizar esta função, mas sim implementá-la manualmente (para compreender o que está ocorrendo nos bastidores). Para realizar este exercício, você deve preencher as lacunas da função escrita no código em anexo. Esta função tem como entrada:

.uma imagem (img),

.o ângulo de rotação (angulo),

.o centro de rotação (centro),

e devolve a imagem rotacionada.

In [2]:
import cv2 as cv
import sys
import numpy as np
from numpy.linalg import inv

In [31]:
def my_rotation(img, angulo, centro):

    # nessa primeira parte, vamos definir a transformação que leva a posicao dos pixels da imagem original
    # para a posicao dos pixels do imagem rotacionada.

    x_centro = centro[0]
    y_centro = centro[1]
    cos_angulo = np.cos(angulo)
    sin_angulo = np.sin(angulo)

    # a primeira matriz de translação muda a origem das coordenadas do canto da imagem para o centro da imagem
    matriz_translacao = np.array([
        [1, 0, -x_centro],
        [0, 1, -y_centro],
        [0, 0, 1],
    ])
    volta_matriz_translacao = np.array([
        [1, 0, x_centro],
        [0, 1, y_centro],
        [0, 0, 1],
    ])
    # a matriz de rotacao aplica a rotacao em torno da origem
    matriz_rotacao = np.array([
        [cos_angulo, -sin_angulo, 0],
        [sin_angulo, cos_angulo, 0],
        [0, 0, 1],
    ])
    # a composicao coloca todas as matrizes em uma só: aplica a translacao (muda a origem), rotaciona, volta para a origem anterior
    matriz_composicao = volta_matriz_translacao @ matriz_rotacao @ matriz_translacao
    # matriz_composicao = matriz_translacao

    # criar imagem rotacionada em preto, com mesmas dimensões da original
    height, width = img.shape[:2]
    rotated_image = np.zeros((height,width,3), np.uint8)
    # o próximo passo é percorrer cada pixel da nova imagem e verificar qual é o pixel correspondente na imagem original
    m_comp_inv = inv(matriz_composicao)
    for linr in range(height):
        for colr in range(width):
            pos_rot = np.matrix([linr, colr, 1]).T
            pos_orig = m_comp_inv @ pos_rot
            lin = round(pos_orig[0,0]); col = round(pos_orig[1,0]);
            if (lin >=0 and lin < height) and (col >= 0 and col < width):
                #opa, é um pixel pertencente à imagem original...
                rotated_image[linr, colr] = img[lin, col]

    return rotated_image

In [32]:
img = cv.imread("Imagens/Palazzo_Farnese_Fassade.jpg")
height, width = img.shape[:2]
centro = (height/2, width/2)
angulo = np.pi/6
my_rotated_image =  my_rotation(img, angulo, centro)

In [ ]:
cv.imshow('Display window', my_rotated_image)
cv.waitKey(0); cv.destroyAllWindows()

Você pode comparar seu resultado com o função do Open CV.

In [27]:
# Obter o centro da imagem
height, width = img.shape[:2]
center = (width/2, height/2)

# Definir a matriz de rotação
M = cv.getRotationMatrix2D(center, 30, 1)

# Aplicar a rotação na imagem
rotated_img = cv.warpAffine(img, M, (width, height))

cv.imshow('Display window', rotated_img)
cv.waitKey(0); cv.destroyAllWindows()

# **Transformação Projetiva de uma Imagem**
Escreva uma função que tenha uma como entrada:

. uma imagem (img),

. uma matriz de uma transformação projetiva (T),

e devolva a imagem transformada.

A ideia é semelhante ao exercício 1, mas ao invés da rotação, temos uma transformação projetiva (homografia).

Teste a sua função, faça a transformação projetiva com apenas um ponto de fuga no eixo $x$ (digamos, o ponto $(2000,0)$ em coordenadas $x,y$), da imagem do Palazzo Farnese Fassade (em anexo).

Você pode usar outra imagem de sua preferência, caso queira. Não se esqueça de trabalhar com coordenadas homogênas.

In [9]:
T = np.array([
	[1, 0, 2],
	[0, 1, 2],
	[1/2000, 0, 1],
])

import numpy as np
from numpy.linalg import inv

def my_transform(img, T):
    # criar imagem rotacionada em preto, com mesmas dimensões da original
    height, width = img.shape[:2]
    transformed_img = np.zeros((height, width, 3), np.uint8)
    # o próximo passo é percorrer cada pixel da nova imagem e verificar qual é o pixel correspondente na imagem original
    T_inv = inv(T)
    for y in range(height):
        for x in range(width):
            transformed_pos = np.matrix([x, y, 1]).dot(T_inv).T 
            transformed_pos /= transformed_pos[2]
            orig_x, orig_y = int(transformed_pos[0]), int(transformed_pos[1])
            
            if 0 <= orig_x < width and 0 <= orig_y < height:
                # opa, é um pixel pertencente à imagem original...
                transformed_img[y, x] = img[orig_y, orig_x]

    return transformed_img
    
img = cv.imread("Imagens/Palazzo_Farnese_Fassade.jpg")
imagem = my_transform(img, T)
cv.imshow('Display window', imagem)
cv.waitKey(0); cv.destroyAllWindows()

C:\Users\danie\AppData\Local\Temp\ipykernel_13416\968443867.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  orig_x, orig_y = int(transformed_pos[0]), int(transformed_pos[1])


# **Estimação da Transformação Projetiva**
**Crie uma função**

Escreva uma função que tenha uma como entrada:

. Quatro pontos “fonte” no plano projetivo, possivelmente impróprios,

. Quatro pontos “destino” no plano projetivo, possivelmente impróprios.

E devolva a transformação projetiva $T: \mathbb{R}\mathbb{P}^2 \rightarrow \mathbb{R}\mathbb{P}^2$ que leva os pontos “fonte” nos pontos
“destino”.

In [13]:
def my_estimation(lst1, lst2):	
	matriz_cria_H = np.zeros(12, np.uint8)[:, np.newaxis].T
	
	for index, ponto in enumerate(lst1):
		# lado esquerdo da matriz
		matriz_ponto = np.array([
			[ponto[0], ponto[1], 1, 0, 0, 0, 0, 0],
			[0, 0, 0, ponto[0], ponto[1], 1, 0, 0],
			[0, 0, 0, 0, 0, 0, ponto[0], ponto[1]],
		])
		# lado direito da matriz
        # adicionando 0's antes das coordenadas
		for i in range(index):
			zeros = np.zeros(3, np.uint8)[:, np.newaxis]
			matriz_ponto = np.hstack((matriz_ponto, zeros))
		matriz_lateral_ponto = np.array([-(lst2[index][0]), -(lst2[index][1]), -1]).T
		matriz_lateral_ponto = matriz_lateral_ponto[:, np.newaxis]
		matriz_ponto = np.hstack((matriz_ponto, matriz_lateral_ponto))
		# adicionando 0's depois das coordenadas
		for i in range(3 - index):
			zeros = np.zeros(3, np.uint8)[:, np.newaxis]
			matriz_ponto = np.hstack((matriz_ponto, zeros))
		matriz_cria_H = np.vstack((matriz_cria_H, matriz_ponto))

	matriz_cria_H = matriz_cria_H[1:]
	
	vetor_cria_H = np.array([0, 0, -1])
	vetor_cria_H = np.hstack((vetor_cria_H, vetor_cria_H, vetor_cria_H, vetor_cria_H))[:, np.newaxis]
	
	vetor_resolvido = np.linalg.solve(matriz_cria_H, vetor_cria_H)
	vetor_resolvido = vetor_resolvido[0:10]
	return matriz_cria_H

(1, 12)
index
0
(3, 8)
[[ 786  144    1    0    0    0    0    0 -565    0    0    0]
 [   0    0    0  786  144    1    0    0 -244    0    0    0]
 [   0    0    0    0    0    0  786  144   -1    0    0    0]]
(4, 12)
index
1
(3, 8)
[[ 1190   302     1     0     0     0     0     0     0 -1355     0     0]
 [    0     0     0  1190   302     1     0     0     0  -244     0     0]
 [    0     0     0     0     0     0  1190   302     0    -1     0     0]]
(7, 12)
index
2
(3, 8)
[[ 1190   761     1     0     0     0     0     0     0     0 -1355     0]
 [    0     0     0  1190   761     1     0     0     0     0  -685     0]
 [    0     0     0     0     0     0  1190   761     0     0    -1     0]]
(10, 12)
index
3
(3, 8)
[[ 786  908    1    0    0    0    0    0    0    0    0 -565]
 [   0    0    0  786  908    1    0    0    0    0    0 -685]
 [   0    0    0    0    0    0  786  908    0    0    0   -1]]
(13, 12)
[[  786   144     1     0     0     0     0     0  -565     0     

Encontre a transformação projetiva que leva o campo de futebol da imagem
do gol no jogo Vasco contra Flamengo (em anexo) em um retângulo correspon-
dente ao campo de futebol visto de cima em projeção ortogonal (em anexo, mas
fique a vontade para usar outra). Para isso, utilize a sua função my_estimation.

Use esta projeção para desenhar o campo
de futebol visto de cima junto com a posição do Leo Pelé, do goleiro e de um
jogador do Flamengo. Para isso, utilize a sua função my_transform.

In [ ]:
# Escreva seu código aqui


Compare seu resultado com o implementado no OpenCV (cv.warpAffine)

# **EXTRA**
# You are Fake News ... or maybe not.
Muitas vezes, os torcedores apaixonados pelo time desconfiam do resultado do VAR e das decisões de impedimento, como foi o caso do vídeo. Teste sua função na imagem da partida entre Union e Boca Juniors (em anexo) e verifique se o método mostrado no vídeo (https://www.instagram.com/papodeboleiros/reel/CvAhQazvuhu/) produz o mesmo resultado de impedimento que a função implementada (basta fazer a transformação na imagem com as "retas paralelas" do vídeo e verificar se de fato são paralelas). Comente sobre o resultado da comparação. Você acredita que este método seja sempre correto? Caso contrário, forneça um contraexemplo onde ele falhe (se possível, com uma imagem).


In [ ]:
# Escreva seu código aqui

# My VAR-lidation
Utilizando suas funções my\_estimation e my\_transform, faça uma função que tem como entrada:

. Duas imagens, uma do jogo e uma do campo (img1,img2)

. Uma lista de pontos "fonte" e "destino"(lst1,lst2)

. Os pixels  relativos aos dois jogadores que estão sendo analisados (p1,p2)

E devolve a imagem do campo com uma linha marcada de ambos os jogadores, semelhante com as análises do VAR.

Perceba que basta desenhar uma linha (pintando todos os pixels com a mesma coordenada em x da mesma cor) quando a imagem do jogo estiver em projeção ortogonal e depois usar a inversa da matriz de projeção para voltar ao formato original.

Observação: a imagem provavelmente sofrerá perda de qualidade neste processo. Se desejar manter a qualidade, você poderá realizar esse processo apenas em uma imagem com a linha inserida e adicioná-la depois de transformada na imagem original.

In [ ]:
# Escreva seu código aqui